In [1]:
import pymongo
import spacy
import decouple
import glob
import datetime
from PIL import Image
from IPython.display import display
from tqdm.notebook import tqdm
from pytrends.request import TrendReq
import tweepy

config = decouple.AutoConfig(' ')
nlp = spacy.load("en_core_web_lg")

In [2]:
def get_gtrend(kw):
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload([kw], timeframe='now 4-H')
    qrys = pytrends.related_queries()[kw]['rising']['query'].values
    return ' '.join([_q.replace(kw, '').replace('  ', ' ').strip() for _q in qrys])

In [3]:
get_gtrend('photo')

'jindal share price emma watson jk rowling kolkata fort kolkata kolkata  result resizer in kb resizer to pdf converter why did the go to jail joke size reducer poses for boys frame shop near me passport size passport size size tetu online resizer dp gold rates in pakistan today background change online to pdf compressor online resize passport size dimensions pose passport size maker'

In [4]:
def get_mongodb_collection(colname):
    client = pymongo.MongoClient(config('MONGODB_URI'))
    return client.Cluster0[colname]

In [5]:
get_mongodb_collection('pix')

Collection(Database(MongoClient(host=['ac-w0gvyzx-shard-00-02.bhunbl6.mongodb.net:27017', 'ac-w0gvyzx-shard-00-01.bhunbl6.mongodb.net:27017', 'ac-w0gvyzx-shard-00-00.bhunbl6.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-13va1m-shard-0', tls=True), 'Cluster0'), 'pix')

In [8]:
def main():
    gtrend = get_gtrend('photo')
    colpix = get_mongodb_collection('pix')
    pixs = list(colpix.find({ 'feed_id': {'$exists': False}}))
    
    sims = []
    for _p in tqdm(pixs):
        sims.append((_p['hash'], nlp(gtrend).similarity(nlp(_p['labels'] + ' ' + _p['colors']))))

    sims = sorted(sims, key=lambda x: -x[1])
    return sims[-5:]

In [9]:
main()

  0%|          | 0/347 [00:00<?, ?it/s]

[('a197e0b535f42c6018fed919f089418bb2ac5e698989590b05fb2db7f3c8b10e',
  0.44848098364209804),
 ('b057f241cb99b1ee21c0a4bfbda357dffa94d6bc118cbb007f6d9454940fd532',
  0.4417691922876015),
 ('8f7a52ee493dd46140ea14ca96e40a1282ce33858972912c1ec80de9a620f899',
  0.42219683194516167),
 ('e8c6dca9d46e30fd3487d91e59f19de77d980f0a105f7336f79d4642999197ee',
  0.3619973319364168),
 ('5e2c317abb6b562a7ddc5c6023604efbe83090cac7374e8aadcee09f60e5958a',
  0.3142278014232861)]